In [1]:
import networkx as nx
import pickle
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from itertools import combinations, groupby
import datetime
import sys

https://stackoverflow.com/questions/40088042/networkx-get-the-distance-between-nodes


In [2]:
entries = os.listdir('grafi/')

In [3]:
def load_graph(name):
    # directory = os.getcwd()
    # filename1 = directory + "/grafi/graph_V7500_E45000_weighted.pickle"

    # G = pickle.load(open(filename1, 'rb'))
    G = pickle.load(open("grafi/"+name, 'rb'))
    num_of_nodes = G.number_of_nodes()
    num_of_edges = G.number_of_edges()

    s = (num_of_nodes, num_of_edges)
    graph = np.zeros(s) # matrix of zeros

    for source, target in G.edges():
        graph[source][target] = G[source][target]['weight']

    return graph

# Edmonds-Karp algorithm

In [4]:
#Edmonds-Karp Algorithm
def edmonds_karp(C, s, t):
        n = len(C) # C is the capacity matrix
        F = [[0] * n for i in range(n)]
        path = bfs(C, F, s, t)
        while path != None:
            flow = min(C[u][v] - F[u][v] for u,v in path)
            for u,v in path:
                F[u][v] += flow
                F[v][u] -= flow
            path = bfs(C, F, s, t)
        return sum(F[s][i] for i in range(n))

#find path by using BFS
def bfs(C, F, s, t):

        queue = [s]
        paths = {s:[]}
        if s == t:
            return paths[s]
        while queue: 
            u = queue.pop(0)
            for v in range(len(C)):
                    if(C[u][v]-F[u][v]>0) and v not in paths:
                        paths[v] = paths[u]+[(u,v)]

                        last_key = list(paths)[-1]
                        if v == t:
                            return paths[v]
                        queue.append(v)
        return None
    

# source = 0  # LA SORGENTE E' 1
# sink = num_of_nodes-1    # LA DESTINAZIONE E' 7
# max_flow_value = max_flow(graph, source, sink)
# print("Edmonds-Karp algorithm")
# print("max_flow_value is: ", max_flow_value)


# Dinitz Algorithm

In [5]:
#Dinic Algorithm

#build level graph by using BFS
def Bfs(C, F, s, t):  # C is the capacity matrix
        n = len(C)
        queue = []
        queue.append(s)
        global level
        level = n * [0]  # initialization
        level[s] = 1  
        while queue:
            k = queue.pop(0)
            for i in range(n):
                    if (F[k][i] < C[k][i]) and (level[i] == 0): # not visited
                            level[i] = level[k] + 1
                            queue.append(i)
        return level[t] > 0

#search augmenting path by using DFS
def Dfs(C, F, k, cp):
        tmp = cp
        if k == len(C)-1:
            return cp
        for i in range(len(C)):
            if (level[i] == level[k] + 1) and (F[k][i] < C[k][i]):
                f = Dfs(C,F,i,min(tmp,C[k][i] - F[k][i]))
                F[k][i] = F[k][i] + f
                F[i][k] = F[i][k] - f
                tmp = tmp - f
        return cp - tmp

#calculate max flow
#_ = float('inf')
def dinitz(C,s,t):
        n = len(C)
        F = [n*[0] for i in range(n)] # F is the flow matrix
        flow = 0
        while(Bfs(C,F,s,t)):
               flow = flow + Dfs(C,F,s,10000000)
        return flow

# source = 0  # A
# sink = num_of_nodes-1    # F
# print("Dinic's Algorithm")
# max_flow_value = MaxFlow(graph, source, sink)
# print("max_flow_value is", max_flow_value)

# Ford-Fulkerson algorithm

In [6]:
#Ford-Fulkerson Algorithm

#find path by using BFS
def dfs(C, F, s, t):
        stack = [s]
        paths={s:[]}
        if s == t:
                return paths[s]
        while(stack):
                u = stack.pop()
                for v in range(len(C)):
                        if(C[u][v]-F[u][v]>0) and v not in paths:
                                paths[v] = paths[u]+[(u,v)]
                                if v == t:
                                        return paths[v]
                                stack.append(v)
        return None

def ford_fulkerson(C, s, t):
        n = len(C) # C is the capacity matrix
        F = [[0] * n for i in range(n)]
        path = dfs(C, F, s, t)
        while path != None:
            flow = min(C[u][v] - F[u][v] for u,v in path)
            for u,v in path:
                F[u][v] += flow
                F[v][u] -= flow
            path = dfs(C,F,s,t)
        return sum(F[s][i] for i in range(n))


# source = 0  # A
# sink = num_of_nodes-1    # F
# max_flow_value = max_flow(graph, source, sink)
# print("Ford-Fulkerson algorithm")
# print("max_flow_value is: ", max_flow_value)


# Push-relabel algorithm

In [7]:
#push-relabel algorithm

def push_relabel(C, s, t):
     n = len(C) # C is the capacity matrix
     F = [[0] * n for i in range(n)]

     # the residual capacity from u to v is C[u][v] - F[u][v]
     height = [0] * n # height of node
     excess = [0] * n # flow into node minus flow from node
     seen   = [0] * n # neighbours seen since last relabel
     # node "queue"
     nodelist = [i for i in range(n) if i != s and i != t]

     #push operation
     def push(u, v):
         send = min(excess[u], C[u][v] - F[u][v])
         F[u][v] += send
         F[v][u] -= send
         excess[u] -= send
         excess[v] += send

     #relabel operation
     def relabel(u):
         # find smallest new height making a push possible,
         # if such a push is possible at all
         min_height = float('inf')
         for v in range(n):
             if C[u][v] - F[u][v] > 0:
                 min_height = min(min_height, height[v])
                 height[u] = min_height + 1
 
     def discharge(u):
         while excess[u] > 0:
             if seen[u] < n: # check next neighbour
                 v = seen[u]
                 if C[u][v] - F[u][v] > 0 and height[u] > height[v]:
                     push(u, v)
                 else:
                     seen[u] += 1
             else: # we have checked all neighbours. must relabel
                 relabel(u)
                 seen[u] = 0
 
     height[s] = n   # longest path from source to sink is less than n long
     excess[s] = float("inf") # send as much flow as possible to neighbours of source
     for v in range(n):
         push(s, v)
 
     p = 0
     while p < len(nodelist):
         u = nodelist[p]
         old_height = height[u]
         discharge(u)
         if height[u] > old_height:
             nodelist.insert(0, nodelist.pop(p)) # move to front of list
             p = 0 # start from front of list
         else:
             p += 1
     return sum(F[s])

# Esecuzione degli algoritmi

In [8]:
def run_edmonds_karp(graph, source, sink):
    print("Edmonds-Karp algorithm")
    max_flow_value = edmonds_karp(graph, source, sink)
    print("max_flow_value is: ", max_flow_value)

def run_dinic(graph, source, sink):
    print("Dinic's Algorithm")
    max_flow_value = dinitz(graph, source, sink)
    print("max_flow_value is", max_flow_value)

def run_ford_fulkerson(graph, source, sink):
    print("Ford-Fulkerson algorithm")
    max_flow_value = ford_fulkerson(graph, source, sink)
    print("max_flow_value is: ", max_flow_value)

def run_push_relabel(graph, source, sink):
    print("Push-Relabel algorithm")
    max_flow_value = push_relabel(graph, source, sink)
    print("max_flow_value is: ", max_flow_value)

In [ ]:
for i in entries:

    graph_file = i #'graph_V5000_E40000_weighted.pickle'
    print("Grafo corrente: ", graph_file, "\n")
    graph = load_graph(graph_file)
    num_of_nodes = len(graph)
    source = num_of_nodes-2  # A
    sink = num_of_nodes-1    # F

    #push_relabel
    #start = datetime.datetime.now()
    #run_push_relabel(graph, source, sink)
    #end = datetime.datetime.now()
    #delta = end - start
    #print("Tempo di esecuzione: ", delta.total_seconds(), "\n")

    #dinitz
    start = datetime.datetime.now()
    run_dinic(graph, source, sink)
    end = datetime.datetime.now()
    delta = end - start
    print("Tempo di esecuzione: ", delta.total_seconds(), "\n")

    # #ek
    start = datetime.datetime.now()
    run_edmonds_karp(graph, source, sink)
    end = datetime.datetime.now()
    delta = end - start
    print("Tempo di esecuzione: ", delta.total_seconds(), "\n")

    #ff
    #start = datetime.datetime.now()
    #run_ford_fulkerson(graph, source, sink)
    #end = datetime.datetime.now()
    #delta = end - start
    #print("Tempo di esecuzione: ", delta.total_seconds(), "\n")

    print("\n\n")